In [1]:
import os
import random
import traceback
from dotenv import load_dotenv
from pymilvus import connections, utility, Collection, FieldSchema, DataType, CollectionSchema
from sentence_transformers import SentenceTransformer
from pymilvus import MilvusClient
from openai import OpenAI


load_dotenv()
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

MILVUS_HOST = os.getenv("MILVUS_HOST")
MILVUS_PORT = os.getenv("MILVUS_PORT")

DB_NAME = os.getenv("DATABASE_NAME")
COLLECTION_NAME = os.getenv("COLLECTION_NAME")

DIMENSION = 512  # SentenceTransformer all-mpnet-base-v2 的輸出維度


# 注入 embedding
def query_and_response_with_no_cache(host_name, port, db_name, collection_name, user_query):
    print(f"{host_name}/{port}/{db_name}/{collection_name}/{user_query}")
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    search_params = {"metric_type": "COSINE", "param": {"nprobe": 16}}
    xq = embedder.encode(user_query)

    client = None
    try:
        client = MilvusClient(
            uri=f"http://{host_name}:{port}",
            db_name=db_name
        )
        res=client.search(
                collection_name=collection_name,
                anns_field="embedding",
                data=[xq],
                limit=1,
                search_params=search_params,
                output_fields=["id","text"]
            )
       # print(res)
        if res and res[0] and res[0][0] and res[0][0]["entity"]:
            #data_id=(res[0][0]["id"])
            #raw_response_material_data=client.get(collection_name=collection_name, ids=[data_id], output_fields=["text"])
    
            prompt = f"""使用以下上下文回答問題：
            {res[0][0]["entity"]["text"]}
    
            問題：{user_query}
            """
            response = openai_client.chat.completions.create(model="gpt-4o-mini", 
            messages=[{"role": "user", "content": prompt}], temperature=0)
            return response.choices[0].message.content.strip()
        else:
            return "找不到相關資訊"
    except Exception as e:
        traceback.print_exc()
        return "發生錯誤"
    finally:
        if client:
            client.close()

/home/ec2-user/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
result=query_and_response_with_no_cache(MILVUS_HOST, MILVUS_PORT, DB_NAME, COLLECTION_NAME, user_query="管理委員會之職務是什麼")
print(result)

127.0.0.1/19530/law_database/law_text_embeddings/管理委員會之職務是什麼
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 620 ms, sys: 95.5 ms, total: 716 ms
Wall time: 7.32 s


In [3]:
from gptcache.adapter.api import init_similar_cache
from gptcache import cache, Config, Cache
from gptcache.adapter.api import put, get, init_similar_cache
from gptcache.processor.post import nop
from gptcache.processor.pre import get_prompt

global_semantic_cache=None
def init_global_semantic_cache():
    global global_semantic_cache
    if not global_semantic_cache:
        print("semantic cache initialized..")
        global_semantic_cache = Cache()
        init_similar_cache( ## FAISS + SQLite
            cache_obj=global_semantic_cache, post_func=nop, config=Config(similarity_threshold=0.9), data_dir="./20250201_01/"
        )

def query_and_response_with_cache(host_name, port, db_name, collection_name, user_query):
    print(f"{host_name}/{port}/{db_name}/{collection_name}/{user_query}")
    global global_semantic_cache
    init_global_semantic_cache()
    try:
        resp=get(user_query, cache_obj=global_semantic_cache, top_k=1)
        if resp and resp[0]:
            return resp[0]
    except Exception as e:
        pass
   
    embedder = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    xq = embedder.encode(user_query)
    search_params = {"metric_type": "COSINE", "param": {"nprobe": 16}}

    client = None
    try:
        client = MilvusClient(
            uri=f"http://{host_name}:{port}",
            db_name=db_name
        )
        res=client.search(
                collection_name=collection_name,
                anns_field="embedding",
                data=[xq],
                limit=1,
                search_params=search_params,
                output_fields=["id","text"]
                )
        
        if res and res[0] and res[0][0] and res[0][0]["entity"]:
            #data_id=(res[0][0]["id"])
           # raw_response_material_data=client.get(collection_name=collection_name, ids=[data_id], output_fields=["text"])
    
            prompt = f"""使用以下上下文回答問題：
            {res[0][0]["entity"]["text"]}
    
            問題：{user_query}
            """
            response = openai_client.chat.completions.create(model="gpt-4o-mini", 
            messages=[{"role": "user", "content": prompt}], temperature=0)
            put(user_query, response.choices[0].message.content.strip(), cache_obj=global_semantic_cache)
            return response.choices[0].message.content.strip()
        else:
            return "找不到相關資訊"
    except Exception as e:
        traceback.print_exc()
        return "發生錯誤"
    finally:
        if client:
            client.close()

In [4]:
%%time
result=query_and_response_with_cache(MILVUS_HOST, MILVUS_PORT, DB_NAME, COLLECTION_NAME, user_query="管理委員會之職務是什麼")
print(result)

127.0.0.1/19530/law_database/law_text_embeddings/管理委員會之職務是什麼
semantic cache initialized..
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 7.65 s, sys: 294 ms, total: 7.94 s
Wall time: 12.3 s


In [5]:
%%time
result=query_and_response_with_cache(MILVUS_HOST, MILVUS_PORT, DB_NAME, COLLECTION_NAME, user_query="管理委員會之職務是什麼")
print(result)

127.0.0.1/19530/law_database/law_text_embeddings/管理委員會之職務是什麼
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 3.13 s, sys: 2.29 ms, total: 3.14 s
Wall time: 1.61 s


In [6]:
%%time
result=query_and_response_with_cache(MILVUS_HOST, MILVUS_PORT, DB_NAME, COLLECTION_NAME, user_query="管理委員會職務包含什麼項目")
print(result)

127.0.0.1/19530/law_database/law_text_embeddings/管理委員會職務包含什麼項目
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 3.05 s, sys: 10.7 ms, total: 3.06 s
Wall time: 1.59 s


In [7]:
%%time
result=query_and_response_with_cache(MILVUS_HOST, MILVUS_PORT, DB_NAME, COLLECTION_NAME, user_query="條例用辭定義")
print(result)

127.0.0.1/19530/law_database/law_text_embeddings/條例用辭定義
管理委員會的職務包括以下幾項：

1. 執行區分所有權人會議的決議事項。
2. 負責共有及共用部分的清潔、維護、修繕及一般改良。
3. 確保公寓大廈及其周圍的安全及環境維護。
4. 提出住戶共同事務的改進建議。
5. 制止住戶的違規行為並提供相關資料。
6. 協調住戶違反相關規定的情況。
7. 管理收益、公共基金及其他經費的收支、保管及運用。
8. 保管規約、會議紀錄、使用執照謄本、竣工圖說等相關文件。
9. 委任、僱傭及監督管理服務人。
10. 提出及公告會計報告、結算報告及其他管理事項。
11. 點收及保管共用部分及其附屬設施設備。
12. 申報公共安全檢查與消防安全設備檢修，並執行改善措施。
13. 處理其他依本條例或規約所定的事項。
CPU times: user 3.08 s, sys: 0 ns, total: 3.08 s
Wall time: 1.59 s


-----

# References
- https://www.restack.io/p/gptcache-knowledge-gptcache-milvus-cat-ai
- https://community.aws/content/2juMSXyaSX2qelT4YSdHBrW2D6s/bridging-the-efficiency-gap-mastering-llm-caching-for-next-generation-ai-part-2
- https://github.com/zilliztech/GPTCache
- https://milvus.io/docs
- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html